# Simulation of 1D Shallow Water Equations

## Numerical Solution using Finite Element Method

### Input Parameters

Input parameters are crucial in programming and data analysis for several reasons:

1. **Flexibility**: They allow functions and programs to be more flexible and reusable. By passing different input parameters, the same function can perform a variety of tasks without modification.

2. **Modularity**: Input parameters help in breaking down complex problems into smaller, manageable pieces. Each function can focus on a specific task, making the code easier to understand and maintain.

3. **Scalability**: They enable the handling of different data sets or conditions without changing the core logic of the program. This is particularly important in data analysis, where the same analysis might need to be applied to different data sets.

4. **Testing and Debugging**: With input parameters, it is easier to test and debug functions. You can pass different values to the function to see how it behaves, which helps in identifying and fixing issues.

5. **Interoperability**: In collaborative environments, input parameters allow different parts of a system to interact seamlessly. Different modules or components can call functions with the required parameters, ensuring smooth integration.

Overall, input parameters are a fundamental concept that enhances the functionality, efficiency, and maintainability of code.

In [1]:
# input parameters

# Physical domain 
x_initial = 0 # (m) initial domain coordinate
x_final = 10 # (m) final domain coordinate

# DG method
N_elements= 16 # number of elements
p_basis_order = 2 # lagrange basis order

# simulation time
n_steps = 4000 # number of time steps
t_limit = 4 # (s) simulation time limit

# Gauss cuadrature
n_gauss_poins = 20 

# plotting setting
plot_every_steps = 400

# PDE evolution method # 0: euler, 1: rk4
evolution_method = 1

In [2]:
import numpy as np

# import inputs
import grid_generation
import basis
import initial_conditions
import evolve 
import integrator
import test

# running some test
test.test_lagrange_basis()
test.test_lagrange_basis_derivative()
test.test_integration()
# test.test_mass_matrix()

# creating mesh
nodes_coordinates_phys_space = grid_generation.generate_1d_mesh(x_initial, x_final, N_elements, p_basis_order)

element_number = np.arange(N_elements)
element_lengths = nodes_coordinates_phys_space[:,-1] - nodes_coordinates_phys_space[:,0]

# generating reference space information
gauss_weights, basis_values_at_gauss_quad, basis_values_x_derivative_at_gauss_quad, basis_values_at_nodes = basis.generate_reference_space(nodes_coordinates_phys_space, n_gauss_poins)

# generating initial conditions
h, u = initial_conditions.generate_initial_conditions(nodes_coordinates_phys_space)

# compute entropy : integral 0.5 * ( g * h**2 + h * u ) dx
entropy = evolve.compute_entropy(element_lengths, gauss_weights, basis_values_at_gauss_quad, h, u)

# compute mass matrix M_ij = integral phi_i(x) phi_j(x) dx and return the inverse matrix of M_ij
mass_matrix_inverse = evolve.compute_mass_matrix_inverse(element_lengths, gauss_weights, basis_values_at_gauss_quad)

# time step
time_step = np.array(t_limit/n_steps) 

# evolving in time the PDE
for number_of_t_step in np.arange(n_steps):

    # If true using euler method
    if evolution_method==0:
        h, u = integrator.euler_method( h, u, time_step, element_number, basis_values_at_gauss_quad, basis_values_x_derivative_at_gauss_quad, gauss_weights, element_lengths, basis_values_at_nodes, mass_matrix_inverse)

    # If true using RK4 method
    if evolution_method==1:
        h, u = integrator.rk4_method( h, u, time_step, element_number, basis_values_at_gauss_quad, basis_values_x_derivative_at_gauss_quad, gauss_weights, element_lengths, basis_values_at_nodes, mass_matrix_inverse)

    # saving the data
    if (number_of_t_step+1) % plot_every_steps == 0:
        print(f'Number of time step : {number_of_t_step+1} / {n_steps}')

        # compute entropy : integral 0.5 * ( g * h**2 + h * u ) dx
        entropy = evolve.compute_entropy(element_lengths, gauss_weights, basis_values_at_gauss_quad, h, u)

        # writing data
        integrator.write_data_file(nodes_coordinates_phys_space,entropy,h,u,False,number_of_t_step+1, t_limit, n_steps)

print(f'Done')

Generating mesh 
Physical domain: [0,10] meters
Number of elements: 16
Nodes per element: 3
Lagrange basis order: 2
Saving initial conditions
Number of time step : 400 / 4000
Writing step 400 | t = 0.4
Number of time step : 800 / 4000
Writing step 800 | t = 0.8
Number of time step : 1200 / 4000
Writing step 1200 | t = 1.2
Number of time step : 1600 / 4000
Writing step 1600 | t = 1.6
Number of time step : 2000 / 4000
Writing step 2000 | t = 2.0
Number of time step : 2400 / 4000
Writing step 2400 | t = 2.4
Number of time step : 2800 / 4000
Writing step 2800 | t = 2.8
Number of time step : 3200 / 4000
Writing step 3200 | t = 3.2
Number of time step : 3600 / 4000
Writing step 3600 | t = 3.6
Number of time step : 4000 / 4000
Writing step 4000 | t = 4.0
Done
